Рекомендательные системмы являются одними из самых популярных приложений DS. Они используются для прогнозирования предпочтений,  которые пользователь поставит какому-нибудь товару в магазине. Amazon использует их, чтобы предлагать продукты клиентам, YouTube решает какое видео воспроизводить следующим, а Facebook рекомендует посты. Существуют также рекомендательные системы для таких доменов, как рестораны, фильмы и онлайн-знакомства. Более того, для некоторых компаний, таких как Netflix, бизнес-модель и ее успех зависят от эффективности их рекомендаций. Netflix даже предложил миллион долларов в 2009 году каждому, кто сможет улучшить его систему рекомендаций на 10%.

Вообще говоря, рекомендательные системы могут быть поделены на 3 типа:
* Базовый подход: предлагайте общие рекомендации каждому пользователю в зависимости от популярности фильма и / или жанра. Например, Кинопоиск 250.
* Content-based рекомендательные системы. Эта система использует метаданные элемента, например жанр, режиссер, описание, актеры и т.д. для фильмов. Общая идея, лежащая в основе этих рекомендательных систем, заключается в том, что если человеку нравится конкретный предмет, ему также понравится предмет, похожий на него. Алгоритм будет использовать метаданные прошлых элементов пользователя. Хорошим примером может служить YouTube, где на основе вашей истории он предлагает вам новые видео, которые вы потенциально можете посмотреть.
* Коллаборативная фильтрация. Эти системы пытаются предсказать рейтинг, который пользователь поставит элементу, на основе прошлых оценок и предпочтений других пользователей. Коллаборативной фильтрации не требуются метаданные элемента.

# Простые рекомендации

* Сначала нужно определить метрику, по которой будем сортировать фильмы
* Затем посчитать метрику для каждого фильма
* Отсортировать фильмы, и выбрать топ результатов

Для работы будем использовать популярный датасет MovieLens. Датасет можно скачать по [ссылке](https://www.kaggle.com/rounakbanik/the-movies-dataset). 

In [1]:
import pandas as pd
import numpy as np

metadata = pd.read_csv('archive/movies_metadata.csv', low_memory=False)
metadata.head(3)

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0


In [2]:
metadata.shape

(45466, 24)

Одна из базовых метрик - это рейтинг. Однако он не принимает во внимание популярность фильма. Таким образом, фильм с рейтингом 9 от  10 пользователей будет считаться «лучше», чем фильм с рейтингом 8,9 от 10 000 пользователей. Поэтому стоит использовать взвешенный рейтинг - учитывающий средний рейтинг и количество голосов.

$$WeightedRating(WR) = \frac{v}{v+m}*R + \frac{m}{v+m}*C$$

* v - количество голосов за фильм

* m - минимальное количество голосов, необходимое для внесения в таблицу

* R - средний рейтинг фильма

* C - средний голос по всем фильмам

m - гиперпараметр. Выберем для него 90 перцентиль - то есть для попадания в таблицу, у фильма должно быть больше голосов, чем у 90 процентов других фильмов.

In [3]:
C = metadata['vote_average'].mean()
print(C)

5.618207215133889


In [4]:
m = metadata['vote_count'].quantile(0.90)
print(m)

160.0


In [5]:
q_movies = metadata.copy().loc[metadata['vote_count'] >= m]
q_movies.shape

(4555, 24)

In [6]:
def weighted_rating(x, m=m, C=C):
    ### функция, которая считает взвешенный рейтинг элемента###
    
    v = x['vote_count'] 
    R = x['vote_average']
    return v / (v + m) * R + m / (v + m) * C

In [7]:
q_movies['score'] = q_movies.apply(weighted_rating, axis=1)

In [8]:
q_movies = q_movies.sort_values('score', ascending=False)

In [9]:
q_movies[['title', 'vote_count', 'vote_average', 'score']].head(10)

,title,vote_count,vote_average,score
314,The Shawshank Redemption,8358.0,8.5,8.445869
834,The Godfather,6024.0,8.5,8.425439
10309,Dilwale Dulhania Le Jayenge,661.0,9.1,8.421453
12481,The Dark Knight,12269.0,8.3,8.265477
2843,Fight Club,9678.0,8.3,8.256385
292,Pulp Fiction,8670.0,8.3,8.251406
522,Schindler's List,4436.0,8.3,8.206639
23673,Whiplash,4376.0,8.3,8.205404
5481,Spirited Away,3968.0,8.3,8.196055
2211,Life Is Beautiful,3643.0,8.3,8.187171


Ну вот, мы построили простую рейтинговую таблицу на основании взвешенного рейтинга фильмов. Смахивает на кинопоиск, правда?

# Content-based рекомендации

Идея content-based подхода заключается в том, чтобы по истории действий пользователя создать для него вектор его предпочтений в пространстве предметов и рекомендовать товары, близкие к этому вектору.

Попробуем рекомендовать фильмы, похожие на конкретный фильм. Для этого вычислим попарные косинусные расстояния для всех фильмов на основе их сюжетных описаний и порекомендуем фильмы на основе этой метрики.

<img src="content.png">

In [10]:
metadata['overview'].head()

0    Led by Woody, Andy's toys live happily in his ...
1    When siblings Judy and Peter discover an encha...
2    A family wedding reignites the ancient feud be...
3    Cheated on, mistreated and stepped on, the wom...
4    Just when George Banks has recovered from his ...
Name: overview, dtype: object

Невозможно вычислить сходство между любыми двумя обзорами в их необработанном виде, поэтому вычислим векторы слов. Как следует из названия, векторы слов - это векторизованное представление слов в документе. Векторы несут семантическое значение. Например, мужчина и король будут иметь векторные представления близко друг к другу, в то время как мужчина и женщина будут иметь представления далеко друг от друга.

Мы вычислим Term Frequency-Inverse Document Frequency (TF-IDF) для каждого документа. Это даст нам матрицу, в которой каждый столбец представляет слово в обзорном словаре (все слова, которые встречаются хотя бы в одном документе), а каждый столбец представляет фильм.

По сути, оценка TF-IDF - это частота встречаемости слова в документе, взвешенная с уменьшением числа документов, в которых оно встречается. Это сделано для уменьшения важности слов, которые часто встречаются в обзорах сюжетов, и, следовательно, их значимости при вычислении окончательной оценки сходства.

TfIdfVectorizer уже реализован в библиотеке scikit. Таким образом, нам нужно проделать следующие шаги:

* импортировать Tfidf из scikit-learn
* убрать стоп-слова, так как они не несут полезной информации
* Замените пропущенные значения пустой строкой
* построить матрицу TF-IDF


In [11]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(stop_words = 'english',dtype = np.float32,max_features = 40000) #объект TF-IDF Vectorizer

metadata['overview'] = metadata['overview'].fillna('') #все пропущенные значения в колонке пустой строкой

tfidf_matrix = tfidf.fit_transform(metadata['overview'])

tfidf_matrix.shape

(45466, 40000)

In [12]:
tfidf.get_feature_names()[5000:5010]

/Users/mark/Library/Python/3.8/lib/python/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


['bruges',
 'bruised',
 'bruiser',
 'bruises',
 'brummel',
 'brummell',
 'brunch',
 'brundle',
 'brunelle',
 'bruner']

Таким образом, у нас получается 75827 различных слов в нашем наборе данных. С помощью этой матрицы мы можем вычислить оценку сходства между двумя фильмами. Мы будем использовать cosine similarity.

$$ \cos(x,y) = \frac{\mathbf{x} \cdot \mathbf{y}}{\|\mathbf{x}\| \|\mathbf{y}\|} = \frac{\sum\limits_{i=1}^{n} x_i y_i}{\sqrt{\sum\limits_{i=1}^{n} x_i^2} \sqrt{\sum\limits_{i=1}^{n} y_i^2}} $$

Поскольку мы использовали TF-IDF, вычисление скалярного произведения между каждым вектором напрямую даст вам оценку косинусного сходства. Следовательно, можно использовать linear_kernel() sklearn вместо cosine_similarities(), поскольку он быстрее. Мы получим матрицу размера 45466x45466, где каждый фильм будет вектором-столбцом 1x45466.

In [13]:
from sklearn.metrics.pairwise import linear_kernel

cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

In [14]:
cosine_sim.shape

(45466, 45466)

In [15]:
cosine_sim[1]

array([0.01504121, 1.0000001 , 0.04681953, ..., 0.        , 0.02230244,
       0.00982309], dtype=float32)

Нужно определить функцию, которая принимает на вход название фильма и выводит список из 10 наиболее похожих фильмов. Нужен механизм для определения индекса фильма в metadata по его названию.

In [16]:
indices = pd.Series(metadata.index, index=metadata['title']).drop_duplicates()

In [17]:
indices[:10]

title
Toy Story                      0
Jumanji                        1
Grumpier Old Men               2
Waiting to Exhale              3
Father of the Bride Part II    4
Heat                           5
Sabrina                        6
Tom and Huck                   7
Sudden Death                   8
GoldenEye                      9
dtype: int64

Функция должна выполнять следующие действия:

* Получить индекс фильма по его названию.

* Получить список оценок косинусного сходства для этого конкретного фильма со всеми фильмами. Преобразуйте его в список кортежей, где первый элемент - это его позиция, а второй - оценка сходства.

* Отсортируйте вышеупомянутый список кортежей на основе оценок сходства (второй элемент).

* Получите 10 лучших элементов этого списка. Игнорируйте первый элемент, так как он относится к себе (фильм, наиболее похожий на себя - это он сам).

* Верните заголовки, соответствующие индексам верхних элементов.

In [18]:
def get_recommendations(title, cosine_sim=cosine_sim):
    
    idx = indices[title] #Получите индекс по названию

    sim_scores = cosine_sim[idx] # Список оценок сходства для фильма по его индексу из матрицы оценок

    movie_indices = sim_scores.argsort()[::-1][1:11] #Отсортируйте массив по скорам (второй элемент) # Возьмите 10 первых элементов (кроме самого фильма) #Получите массив индексов этих 10 элементов 

    # Верните названия топ10 похожих фильмов
    return metadata['title'].iloc[movie_indices]

In [19]:
get_recommendations('The Dark Knight Rises')

12481                                      The Dark Knight
150                                         Batman Forever
1328                                        Batman Returns
585                                                 Batman
15511                           Batman: Under the Red Hood
21194    Batman Unmasked: The Psychology of the Dark Kn...
9230                    Batman Beyond: Return of the Joker
19792              Batman: The Dark Knight Returns, Part 1
18035                                     Batman: Year One
3095                          Batman: Mask of the Phantasm
Name: title, dtype: object

# Коллаборативная фильтрация

<img src="collaborative.png">

Помимо метаданных фильмов, у нас есть еще один ценный источник информации: данные о рейтингах пользователей. Наша система рекомендаций может порекомендовать фильм, похожий на «Начало (2010)», на основе оценок пользователей. Другими словами, какие еще фильмы получили аналогичные оценки других пользователей? Это был бы пример коллаборативной фильтрации item-item. Таким примером является рекомендация по типу «Пользователям, которым понравился этот элемент, понравились и другие». Мы будем исследовать набор данных ratings.csv, и сформируем векторы оценок пользователей.

В файле links.csv лежит маппинг разных id фильмов. В частности в ratings.csv (который мы будем использовать) используется movieId. В уже знакомом нам metadata используется tmdbId. Чтобы использовать обе таблички, добавим в metadata колонку с movieId - которая по сути является маппингом на колонку metadata['id'] с помощью links.

In [20]:
links = pd.read_csv('archive/links.csv')

links = links.dropna()

links = links.drop_duplicates(subset=["tmdbId"],keep='first')

id_to_movieId = dict(links[['tmdbId','movieId']].values)

metadata['id'] = pd.to_numeric(metadata['id'],errors='ignore')

metadata = metadata.dropna(subset=['id'])

metadata['movieId'] = metadata['id'].map(id_to_movieId)

In [21]:
#Создадим словарь отображения названия фильма в его movieId
title_to_id = dict(zip(metadata.title.tolist(), metadata.movieId.tolist()))

В вашем распоряжении есть два файла - ratings.csv и ratings_small.csv. Второй файлик содержит намного меньше информации, но для быстрой работы давайте использовать его (хотя качество, конечно же, получится хуже). При желании вы можете обучить модель на полном файле рейтингов (ratings.csv).

In [22]:
ratings = pd.read_csv('archive/ratings_small.csv')

In [23]:
ratings.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [24]:
ratings_f = ratings[ratings.groupby('userId')['userId'].transform('count') >= 50]#Отфильтруйте таблицу - оставьте только те userId, которые проставили не менее 50 оценок

ratings_pivot = ratings_f.pivot(index='movieId', columns='userId', values='rating')#Создайте pivot таблицу из отфильтрованной ratings_f. Индексы - фильмы, колонки - пользователи, значения - рейтинг

In [25]:
ratings_pivot.head()

userId,2,3,4,5,7,8,12,13,15,17,...,655,656,658,659,660,662,664,665,667,671
movieId,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,NaN,NaN,3.0,NaN,NaN,5.0,2.0,NaN,...,NaN,NaN,NaN,NaN,2.5,NaN,3.5,NaN,NaN,5.0
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,...,4.0,NaN,NaN,NaN,NaN,5.0,NaN,3.0,NaN,NaN
3,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.5,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN


##### Matrix Factorization.

Что ж, теперь в нашем распоряжении есть матрица оценок пользователь-айтем. Из математики мы знаем, что любую матрицу можно разложить на произведение трех матриц - например алгоритмом SVD. Но матрицы оценок очень разрежены, 99 % — обычное дело. А SVD не знает, что такое пропуски. Заполнять их средним значением не очень хочется. И в целом, нас не очень интересует матрица сингулярных значений — мы просто хотим получить скрытое представление пользователей и предметов, которое при перемножении будет приближать истинный рейтинг. Можно сразу раскладывать на две матрицы.

<img src="svd.png">

Что же делать с пропусками? Забить на них. Оказалось, что можно успешно обучать приближать рейтинги по метрике RMSE с помощью SGD или ALS, вообще игнорируя пропуски. Первый такой алгоритм — Funk SVD, который придумали в 2006 году в ходе решения соревнования от Netflix.

Но например в задаче рекомендации товаров, мы имеем уже не матрицу оценок, а матрицу некоторых событий. Она будет состоять в основном из нулей и единичек, иногда каких то чисел побольше. Таким образом, у  нас будут присутствовать только положительные примеры. У нас нет примеров товаров, которые человек никогда не купит - получается мы не можем понять, человек не видел товар или он ему не нравится. Таким образом фидбэк от пользователя может быть двух типов:

* Explicit feedback - есть положительные и отрицательные примеры.
* Implicit feedback - есть только положительные.

Так вот, забить на пропуски получается только в случае задачи explicit feedback. В случае implicit можно заполнить пропущенные значения например нулем, и настроить веса в оптимизируемом функционале - низкие для нулей, и повыше для ненулевых ячеек.

Для того чтобы лучше разобраться в математике, можно прочитать например вот [этот пост](https://habr.com/ru/company/yandex/blog/241455/).

<img src="netflix.png">

Алгоритм SVD реализован в библиотеке [Surprise](https://github.com/NicolasHug/Surprise). Далее мы обучим модель и используем ее для прогнозирования рейтингов фильмов, которые данный пользователь, например с 𝑖𝑑 = 2, еще не получил оценку.

In [29]:
#%pip install scikit-surprise

In [30]:
from surprise import Dataset, Reader, SVD, accuracy
from surprise.model_selection import train_test_split

reader = Reader(rating_scale=(1, 5))
data = Dataset.load_from_df(ratings_f[['userId','movieId','rating']], reader)

trainset, testset = train_test_split(data, test_size=.25)
algorithm = SVD()
algorithm.fit(trainset)
predictions = algorithm.test(testset)

accuracy.rmse(predictions)

RMSE: 0.8929


0.8928866531597774

Напишем нашу функцию, которая принимает на вход id пользователя, а на выходе предлагает ему топ 10 фильмов, которые он еще не видел.

In [31]:
def pred_user_rating(ui):
    if ui in ratings_f.userId.unique():
        ui_list = ratings_f[ratings_f['userId'] == ui]['movieId'].tolist() #Из таблицы ratings_f создайте list фильмов, которые оценил конкретный пользователь
        d = {k: v for k, v in title_to_id.items() if v not in set(ui_list)} #Создайте инвертированный словарь title_to_id, но только для тех фильмов, которые ui не видел. То есть исключите
        #из списка фильмов множество ui_list. Инвертированный - то есть ключи стали значениями, а значения - ключами
        
        #С помощью нашей обученной модели, проставим предсказанные рейтинги фильмам, которые пользователь еще не видел.
        predictedL = []
        for i, j in d.items():     
            predicted = algorithm.predict(ui, j)
            predictedL.append((i, predicted[3])) 
        pdf = pd.DataFrame(predictedL, columns=['movies', 'ratings']) #Создайте датафрейм из массива predictedL с колонками ['movies', 'ratings']
        
        pdf = pdf.sort_values(by='ratings', ascending=False) #Отсортируйте таблицу по колонке ratings, от большего - к меньшему  
        
        pdf.set_index('movies', inplace=True)    
        return pdf.head(10)        
    else:
        print("Пользователь не найден в списке!")
        return None

In [32]:
user_id = 2
predicted_ratings = pred_user_rating(user_id)

In [33]:
predicted_ratings

,ratings
movies,
Toy Story,3.432342
The Flapper,3.432342
5 to 7,3.432342
Seymour: An Introduction,3.432342
Open Secret,3.432342
Puffball,3.432342
Into Thin Air: Death on Everest,3.432342
Parole de flic,3.432342
War-Time Romance,3.432342
